# Lookout Smishing & Phishing Threat Hunting

This notebook provides threat hunting queries for investigating SMS phishing (smishing) attacks detected by Lookout.

## Prerequisites
- Lookout data connector configured in Microsoft Sentinel
- LookoutEvents parser deployed
- SMISHING_ALERT events ingested

## 1. Smishing Campaign Detection

Identify potential phishing campaigns by analyzing URLs that target multiple users.

In [1]:
LookoutEvents
| where EventType == "SMISHING_ALERT"
| where isnotempty(SmishingAlertURL)
| summarize 
    TargetCount = dcount(DeviceEmailAddress),
    FirstSeen = min(TimeGenerated),
    LastSeen = max(TimeGenerated),
    Categories = make_set(SmishingAlertCategory),
    Severities = make_set(SmishingAlertSeverity)
    by SmishingAlertURL
| where TargetCount >= 2
| sort by TargetCount desc

SmishingAlertURL,TargetCount,FirstSeen,LastSeen,Categories,Severities
https://secure-bank-verify.malicious.com/login,15,2025-01-02T06:30:00Z,2025-01-05T15:20:00Z,"[""CREDENTIAL_THEFT""]","[""CRITICAL""]"
https://delivery-package-update.fake.net/track,12,2025-01-03T10:15:00Z,2025-01-05T12:45:00Z,"[""PHISHING""]","[""HIGH""]"
https://irs-tax-refund.scam.org/claim,8,2025-01-04T08:00:00Z,2025-01-05T14:00:00Z,"[""FINANCIAL_SCAM""]","[""CRITICAL""]"
https://account-suspended.phish.io/verify,5,2025-01-04T14:30:00Z,2025-01-05T09:15:00Z,"[""CREDENTIAL_THEFT""]","[""HIGH""]"
https://free-gift-card.spam.net/claim,3,2025-01-05T07:00:00Z,2025-01-05T11:30:00Z,"[""SPAM"", ""PHISHING""]","[""MEDIUM""]"


## 2. High-Risk Smishing Alerts

Focus on critical and high severity smishing attempts.

In [2]:
LookoutEvents
| where EventType == "SMISHING_ALERT"
| where SmishingAlertSeverity in ("CRITICAL", "HIGH")
| project 
    TimeGenerated,
    DeviceEmailAddress,
    DevicePlatform,
    SmishingAlertType,
    SmishingAlertCategory,
    SmishingAlertSeverity,
    SmishingAlertURL,
    SmishingAlertDescription
| sort by TimeGenerated desc

TimeGenerated,DeviceEmailAddress,DevicePlatform,SmishingAlertType,SmishingAlertCategory,SmishingAlertSeverity,SmishingAlertURL,SmishingAlertDescription
2025-01-05T15:20:00Z,john.smith@contoso.com,ANDROID,SMS_LINK,CREDENTIAL_THEFT,CRITICAL,https://secure-bank-verify.malicious.com/login,Fake banking login page attempting credential theft
2025-01-05T14:00:00Z,sarah.jones@contoso.com,IOS,SMS_LINK,FINANCIAL_SCAM,CRITICAL,https://irs-tax-refund.scam.org/claim,IRS impersonation tax refund scam
2025-01-05T12:45:00Z,mike.wilson@contoso.com,ANDROID,SMS_LINK,PHISHING,HIGH,https://delivery-package-update.fake.net/track,Package delivery phishing attempt
2025-01-05T09:15:00Z,lisa.chen@contoso.com,IOS,SMS_LINK,CREDENTIAL_THEFT,HIGH,https://account-suspended.phish.io/verify,Account suspension phishing lure
2025-01-04T16:30:00Z,david.brown@contoso.com,ANDROID,SMS_LINK,CREDENTIAL_THEFT,CRITICAL,https://secure-bank-verify.malicious.com/login,Fake banking login page attempting credential theft


## 3. Smishing by Category Analysis

Understand the types of smishing attacks targeting your organization.

In [3]:
LookoutEvents
| where EventType == "SMISHING_ALERT"
| summarize 
    Count = count(),
    UniqueTargets = dcount(DeviceEmailAddress)
    by SmishingAlertCategory, SmishingAlertType
| sort by Count desc

SmishingAlertCategory,SmishingAlertType,Count,UniqueTargets
CREDENTIAL_THEFT,SMS_LINK,45,28
PHISHING,SMS_LINK,32,22
FINANCIAL_SCAM,SMS_LINK,18,15
SPAM,SMS_LINK,12,10
MALWARE_DELIVERY,SMS_LINK,8,6


## 4. Users Targeted by Multiple Smishing Attempts

Identify users who may be specifically targeted.

In [4]:
LookoutEvents
| where EventType == "SMISHING_ALERT"
| summarize 
    AttackCount = count(),
    UniqueURLs = dcount(SmishingAlertURL),
    Categories = make_set(SmishingAlertCategory),
    FirstSeen = min(TimeGenerated),
    LastSeen = max(TimeGenerated)
    by DeviceEmailAddress
| where AttackCount >= 3
| sort by AttackCount desc

DeviceEmailAddress,AttackCount,UniqueURLs,Categories,FirstSeen,LastSeen
john.smith@contoso.com,12,8,"[""CREDENTIAL_THEFT"", ""PHISHING"", ""FINANCIAL_SCAM""]",2025-01-01T08:00:00Z,2025-01-05T15:20:00Z
exec.ceo@contoso.com,9,7,"[""CREDENTIAL_THEFT"", ""FINANCIAL_SCAM""]",2025-01-02T10:30:00Z,2025-01-05T14:45:00Z
sarah.jones@contoso.com,7,5,"[""PHISHING"", ""FINANCIAL_SCAM""]",2025-01-02T09:15:00Z,2025-01-05T14:00:00Z
finance.director@contoso.com,6,4,"[""CREDENTIAL_THEFT"", ""FINANCIAL_SCAM""]",2025-01-03T11:00:00Z,2025-01-05T10:30:00Z
mike.wilson@contoso.com,5,4,"[""PHISHING"", ""SPAM""]",2025-01-03T14:20:00Z,2025-01-05T12:45:00Z


## 5. Smishing Trend Analysis

Track smishing attack trends over time.

In [5]:
LookoutEvents
| where EventType == "SMISHING_ALERT"
| summarize Count = count() by bin(TimeGenerated, 1d), SmishingAlertCategory
| render timechart

Time Chart rendered - Smishing attack trends by category over the past week